In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("Categorical_train.csv")

In [3]:
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [4]:
df.ord_2.value_counts()

Freezing       99816
Lava Hot       63908
Boiling Hot    60627
Cold           33768
Hot            22227
Warm           19654
Name: ord_2, dtype: int64

In [6]:
mapping = {
    "freezing": 0,
    "Warm": 1,
    "Cold": 2,
    "Boiling Hot": 3,
    "Hot": 4,
    "Lava Hot": 5
}

In [7]:
df.loc[:,"ord_2"] = df.ord_2.map(mapping)

In [8]:
df.ord_2.value_counts()

5.0    63908
3.0    60627
2.0    33768
4.0    22227
1.0    19654
Name: ord_2, dtype: int64

In [9]:
from sklearn import preprocessing

In [10]:
df1 = pd.read_csv("Categorical_train.csv")

In [12]:
df1.isnull().sum()

id        0
bin_0     0
bin_1     0
bin_2     0
bin_3     0
bin_4     0
nom_0     0
nom_1     0
nom_2     0
nom_3     0
nom_4     0
nom_5     0
nom_6     0
nom_7     0
nom_8     0
nom_9     0
ord_0     0
ord_1     0
ord_2     0
ord_3     0
ord_4     0
ord_5     0
day       0
month     0
target    0
dtype: int64

In [13]:
lbl_enc = preprocessing.LabelEncoder()

In [14]:
df1.loc[:, "ord_2"] = lbl_enc.fit_transform(df.ord_2.values)

In [15]:
from sklearn import linear_model
from sklearn import metrics

In [18]:
import pandas as pd
from sklearn import model_selection
if __name__ == "__main__":
 # Read training data
    df3 = pd.read_csv("Categorical_train.csv")
 # we create a new column called kfold and fill it with -1
    df3["kfold"] = -1

 # the next step is to randomize the rows of the data
    df3 = df3.sample(frac=1).reset_index(drop=True)

 # fetch labels
    y = df3.target.values

 # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)

 # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=df3, y=y)):
        df3.loc[v_, 'kfold'] = f

 # save the new csv with kfold column
    df3.to_csv("cat_train_folds.csv", index=False)

In [25]:
def run(fold):
    features = [ f for f in df.columns if f not in ("id", "target", "kfold")] 

    for col in features:
        df.loc[:,col] = df[col].astype(str).fillna("None")

    df_train = df[df.kfold != fold].reset_index(drop=True)
 # get validation data using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)
 # initialize OneHotEncoder from scikit-learn
    ohe = preprocessing.OneHotEncoder()

    full_data = pd.concat([df_train[features], df_valid[features]],axis=0)
    ohe.fit(full_data[features])
 # transform training data
    x_train = ohe.transform(df_train[features])
 # transform validation data
    x_valid = ohe.transform(df_valid[features])
 # initialize Logistic Regression model
model = linear_model.LogisticRegression()

In [26]:
 model.fit(x_train, df_train.target.values)
 # predict on validation data
 # we need the probability values as we are calculating AUC
 # we will use the probability of 1s
 valid_preds = model.predict_proba(x_valid)[:, 1]
 # get roc auc score
 auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
 # print auc

NameError: name 'x_train' is not defined